<a href="https://colab.research.google.com/github/Sts87/FlightOnTime/blob/feature%2Fmodel-training/FlightOnTime-Analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### CARGA DE DATOS

In [2]:
# DATASET
# Airline:  aerolínea
# Flight:   tipo de aeronave
# Time:     hora de salida en minutos (desde medianoche)
# Length:   tiempo de vuelo en minutos
url = 'https://raw.githubusercontent.com/Sts87/FlightOnTime/refs/heads/main/data/Airlines.csv'
df = pd.read_csv(url)

### INSPECCIÓN INICIAL

In [4]:
df.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [5]:
df.shape

(539383, 9)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


### LIMPIEZA DE DATOS

In [9]:
# ELIMINAR COLUMNA CODIGO
df.drop('id', axis=1, inplace=True)

In [10]:
df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
df.replace('', inplace=True)
df.isnull().sum()

/tmp/ipython-input-1900750314.py:2: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df.replace('', inplace=True)


,0
Airline,0
Flight,0
AirportFrom,0
AirportTo,0
DayOfWeek,0
Time,0
Length,0
Delay,0


### ANÁLISIS ESTADÍSTICO

In [11]:
# VARIABLES NÚMERICAS
df.describe()

,Flight,DayOfWeek,Time,Length,Delay
count,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000
mean,2427.928630,3.929668,802.728963,132.202007,0.445442
std,2067.429837,1.914664,278.045911,70.117016,0.497015
min,1.000000,1.000000,10.000000,0.000000,0.000000
25%,712.000000,2.000000,565.000000,81.000000,0.000000
50%,1809.000000,4.000000,795.000000,115.000000,0.000000
75%,3745.000000,5.000000,1035.000000,162.000000,1.000000
max,7814.000000,7.000000,1439.000000,655.000000,1.000000


In [12]:
# VARIABLES CATEGÓRICAS
df.describe(include='object')

,Airline,AirportFrom,AirportTo
count,539383,539383,539383
unique,18,293,293
top,WN,ATL,ATL
freq,94097,34449,34440


### PROPORCIÓN DE CLASES DE LA VARIABLE OBJETIVO

In [ ]:
# IDENTIFICAR SI HAY DESBALANCEO
# No hay desbalanceo: proporcion 55-45%
df_cnt = df['Delay'].value_counts()
df_pct = df['Delay'].value_counts(normalize=True).round(2) * 100

df_delay = pd.concat([df_cnt, df_pct], axis=1)
df_delay

,count,proportion
Delay,,
0,299119,55.0
1,240264,45.0


### NUEVAS VARIABLES

In [ ]:
df['Length'].describe().round(1)

,Length
count,539383.0
mean,132.2
std,70.1
min,0.0
25%,81.0
50%,115.0
75%,162.0
max,655.0


In [ ]:
# Nuevas variables candidatas:
# Time of day
clases = [0, 360, 720, 1080, 1440]
labels = ['Early morning', 'Morning', 'Afternoon', 'Night']
df['Time_day'] = pd.cut(x=df['Time'], bins=clases, labels=labels, include_lowest=True)

# Duration
# : short (<3 hrs), medium (3-6 hrs), long (6-16 hrs), and ultra-long (>16 hrs)
clases = [0, 90, 180, 960]
labels = ['Short', 'Medium', 'Long']
df['Duration'] = pd.cut(x=df['Length'], bins=clases, labels=labels, include_lowest=True)

In [ ]:
# TODO: Value counts en variables, graficos basicos, distribuciones
# length -> distribucion normal

df.sample(5)

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay,Time_day,Duration
45971,US,928,CLT,ORD,5,868,120,1,Afternoon,Medium
118439,9E,4097,MEM,TUL,2,1165,85,0,Night,Medium
231271,DL,1614,FLL,JFK,2,746,179,1,Afternoon,Medium
409739,B6,1416,LGB,AUS,5,720,165,0,Morning,Medium
309737,WN,1874,DEN,SJC,6,1175,165,1,Night,Medium


In [ ]:
df['Duration'].value_counts(normalize=True)

,proportion
Duration,
Medium,0.490540
Short,0.330463
Long,0.178997
